# クラス分類器の不確実性推定

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# ジュピターノートブック上でグラフを表示させるための処理
%matplotlib inline

import mglearn

from sklearn.datasets import load_breast_cancer
from sklearn.datasets import make_moons
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

## 準備

In [3]:
X, y = make_circles(noise=0.25, factor=0.5, random_state=1)

In [4]:
# わかりやすいようにクラスを"blue"と"red"にする
y_named = np.array(["blue", "red"])[y]

In [5]:
# train_test_splitは任意の数の配列に適用できる。
# すべての配列は整合するように分割される。
X_train, X_test, y_train_named, t_test_named, y_train, y_test = train_test_split(X, y_named, y, random_state=0)

In [6]:
# 勾配ブースティングモデルを構築
gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train_named)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## 決定関数（Decision Function）

2クラス分類の場合、decision_functionの結果の配列は(n_samples, )の形になり、サンプルごとに1つの浮動小数点が返される。

In [7]:
print("X_test.shape: {}".format(X_test.shape))
print("Decision function shape: {}".format(gbrt.decision_function(X_test).shape))

X_test.shape: (25, 2)
Decision function shape: (25,)


⬆︎　この値には、あるデータポイントが「陽性」（この場合はクラス1）であると、モデルが信じている度合いがエンコードされている。正であれば陽性クラスを、負であれば「陰性」（つまり陽性以外）クラスを意味する。

In [8]:
# decision_functionの最初のいくつかを表示
print("Decision function:\n{}".format(gbrt.decision_function(X_test)[:6]))

Decision function:
[ 4.13592629 -1.7016989  -3.95106099 -3.62599351  4.28986668  3.66166106]


⬆︎関数の符号だけ見れば、予測クラスがわかる。

正であれば陽性クラスを、負であれば「陰性」

In [9]:
print("Thresholded decision function:\n{}".format(gbrt.decision_function(X_test) > 0))
print("Predictions:\n{}".format(gbrt.predict(X_test)))

Thresholded decision function:
[ True False False False  True  True False  True  True  True False  True
  True False  True False False False  True  True  True  True  True False
 False]
Predictions:
['red' 'blue' 'blue' 'blue' 'red' 'red' 'blue' 'red' 'red' 'red' 'blue'
 'red' 'red' 'blue' 'red' 'blue' 'blue' 'blue' 'red' 'red' 'red' 'red'
 'red' 'blue' 'blue']


2クラス分類では、「陰性」クラスがclasses_属性の第1エントリに、「陽性」クラスが第2エントリになる。完全にpredictと同じ結果を再現したければ、classes_属性を使えばいい

In [10]:
# True/Falseを0/1に
greater_zero = (gbrt.decision_function(X_test) > 0).astype(int)

# 0/1をclasses_のインデックスに使う
pred = gbrt.classes_[greater_zero]

# predはgbrt.predictの出力と同じになる
print("pred is equal to predictions: {}".format(np.all(pred == gbrt.predict(X_test))))

pred is equal to predictions: True


In [11]:
decision_function = gbrt.decision_function(X_test)
print("Decision function minimum: {:.2f} maximum: {:.2f}".format(np.min(decision_function), np.max(decision_function)))

Decision function minimum: -7.69 maximum: 4.29


## 確率の予測

predict_probaの出力は、それぞれのクラスに属する確率で、decision_functionの出力よりも理解しやすい。

出力配列の形は、2クラス分類問題では、常に(n_samples, 2)になる。

In [12]:
print("Shape of probabilitties: {}".format(gbrt.predict_proba(X_test).shape))

Shape of probabilitties: (25, 2)


⬆︎ gbrt.predict_proba()の戻り値の形

各行の第1エントリ（第1列）は第1クラスの予測確率で、第2エントリ（第2列）は第2クラスの予測確率である。

確率なので、predict_probaの出力は常に0から1であり、双方の和は常に1になっている。

In [14]:
# predict_probaの出力(つまり確率)の最初の数行を見る
print("Predited probabilities:\n{}".format(gbrt.predict_proba(X_test[:6])))

Predited probabilities:
[[0.01573626 0.98426374]
 [0.84575649 0.15424351]
 [0.98112869 0.01887131]
 [0.97406775 0.02593225]
 [0.01352142 0.98647858]
 [0.02504637 0.97495363]]


⬆︎2つのクラスの確率の和なので、どちらかが50%以上の確率（確信度）になっており、そのクラスが`予測クラス`になる。